## [결측치 missing value 처리하기]
- 데이터 미입력 또는 데이터 저장 과정에서 지워지거나 깨지는 등 손상된 데이터
- 주로 빈칸
- NaN(Not a Number), NaT(Not a Text)
- Numpy 모듈 NaN/math 모듈 nan 사용

(1) 모듈 로딩하기<hr>

In [1]:
import pandas as pd

(2) 데이터 준비

In [2]:
file = '../DATA/employees.csv'

(3) 저장: csv -> DF

In [4]:
#구분자 = 쉼표, 헤더(첫번째 줄) : 컬럼명.
empDF = pd.read_csv(file)

(4) 데이터 확인

In [6]:
#데이터의 전반적 요약 정리 -> 컬럼별 결측치 여부 확인/처리,  컬럼별 데이터 타입과 실제 데이터 타입 비교하여 다른 경우 형변환. 

empDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   First Name  933 non-null    object 
 1   Gender      854 non-null    object 
 2   Start Date  999 non-null    object 
 3   Salary      999 non-null    float64
 4   Mgmt        933 non-null    object 
 5   Team        957 non-null    object 
dtypes: float64(1), object(5)
memory usage: 47.0+ KB


- 데이터의 컬럼별 분포 확인하기:describe() 컬럼별 기술 통계값

In [8]:
empDF.describe()

,Salary
count,999.000000
mean,90655.528529
std,32939.511615
min,35013.000000
25%,62560.000000
50%,90427.000000
75%,118744.500000
max,149908.000000


In [15]:
#최빈값: mode(), 중앙값: median()

print(empDF.mode(numeric_only = True), empDF.median(numeric_only = True),empDF.mean(numeric_only = True),sep = '\n\n')

     Salary
0   86676.0
1   91462.0
2  121160.0
3  145988.0
4  147183.0

Salary    90427.0
dtype: float64

Salary    90655.528529
dtype: float64


(5) 데이터전처리(preprocessing) : 결측치 처리

- 결측치 확인: isna(), isnull(): 원소 단위로 검사후 결측치인 경우 True 반환
- 

In [21]:
empDF.columns

Index(['First Name', 'Gender', 'Start Date', 'Salary', 'Mgmt', 'Team'], dtype='object')

In [23]:
print(empDF.isna().sum(), empDF.isnull().sum(), sep= '\n\n')

First Name     68
Gender        147
Start Date      2
Salary          2
Mgmt           68
Team           44
dtype: int64

First Name     68
Gender        147
Start Date      2
Salary          2
Mgmt           68
Team           44
dtype: int64


- 결측치 확인 2: notna(), notnull(): 결측치가 아니면 True 반환-> 정상치 몇 개인지 확인 가능

In [25]:
print(empDF.notna().head(), empDF.notnull().head(), sep = '\n\n')

   First Name  Gender  Start Date  Salary  Mgmt   Team
0        True    True        True   False  True   True
1        True    True        True    True  True  False
2        True    True       False    True  True   True
3        True   False        True    True  True   True
4        True    True        True    True  True   True

   First Name  Gender  Start Date  Salary  Mgmt   Team
0        True    True        True   False  True   True
1        True    True        True    True  True  False
2        True    True       False    True  True   True
3        True   False        True    True  True   True
4        True    True        True    True  True   True


- 결측치 처리: 1. 삭제- dropna()

In [26]:
#모든 행에 1개의 값이라도 NaN이면 삭제함. 
empDF2 = empDF.dropna()

print(empDF.shape, empDF2.shape)

(1001, 6) (761, 6)


In [30]:
empDF3 = empDF.dropna(how = 'all') #how=all:행에 모든 데이터(값)가 Na이면 삭제함. 

#특정 컬럼의 na만 체크하여 삭제하는 방법
#성별에 따른 연 분석= NA면 분석 불가능한 컬럼을 지정함.
empDF4 = empDF.dropna(subset = ['Gender', 'Salary'])

In [36]:
#컬럼별 정상 데이터(na가 아닌 데이터)의 갯수를 지정 후 해당 갯수만큼 정상 데이터가 없는 컬럼을 삭제함. 

empDF5 = empDF.dropna(thresh =900 , axis = 'columns') # na가 아닌 데이터가 900개 미만인 컬럼인 'Gender' 삭제
print(empDF5.shape, empDF5.columns)

(761, 0) Index([], dtype='object')


- 결측치 처리 = 대체: fillna()

In [38]:
#Gender 컬럼의 결측값을 뭘로 채울까?
empDF.describe(include = 'all').head(4)

,First Name,Gender,Start Date,Salary,Mgmt,Team
count,761,761,761,761.0,761,761
unique,199,2,745,NaN,2,10
top,Marilyn,Female,5/12/09,NaN,False,Business Dev
freq,10,392,2,NaN,382,88


In [42]:
# column의 최빈값으로 na 채우기
empDF.Gender.mode()[0]
genSR = empDF.Gender.fillna(empDF.Gender.mode()[0])
genSR. isna(). sum()

0

In [43]:
# ->na값 이전과 이후 값으로 채우기:
#method parameter에 ffill(이전값)
empDF.Gender.fillna(method='ffill')

4        Male
5        Male
6      Female
8      Female
9      Female
        ...  
994      Male
996      Male
997      Male
998      Male
999      Male
Name: Gender, Length: 761, dtype: object

In [44]:
#method parameter에 bfill(이후값)
empDF.Gender.fillna(method='bfill')

4        Male
5        Male
6      Female
8      Female
9      Female
        ...  
994      Male
996      Male
997      Male
998      Male
999      Male
Name: Gender, Length: 761, dtype: object

In [51]:
import numpy as np

df = pd.DataFrame([[np.nan, 2, np.nan, 0],
                   [3, 4, np.nan, 1],
                   [np.nan, np.nan, np.nan, np.nan],
                   [np.nan, 3, np.nan, 4]],
                   columns=list("ABCD"))
df

,A,B,C,D
0,NaN,2.0,NaN,0.0
1,3.0,4.0,NaN,1.0
2,NaN,NaN,NaN,NaN
3,NaN,3.0,NaN,4.0


In [54]:
df.A.fillna(method='bfill')

0    3.0
1    3.0
2    NaN
3    NaN
Name: A, dtype: float64

In [55]:
#B컬럼/D컬럼 평균으로 결측치 대체
df.B.fillna(df.B.mean())
df.D.fillna(df.D.mean())

0    0.000000
1    1.000000
2    1.666667
3    4.000000
Name: D, dtype: float64